In [1]:
import privacyraven as pr

from privacyraven.utils.data import get_emnist_data
from privacyraven.extraction.core import ModelExtractionAttack
from privacyraven.utils.query import get_target
from privacyraven.models.victim import train_mnist_victim
from privacyraven.models.pytorch import ImagenetTransferLearning, ThreeLayerClassifier
#from pl_bolts.callbacks import PrintTableMetricsCallback

#callback = PrintTableMetricsCallback()

# Create a query function for a target PyTorch Lightning model
model = train_mnist_victim()


def query_mnist(input_data):
    # PrivacyRaven provides built-in query functions
    return get_target(model, input_data, (1, 1, 28, 28))

# Obtain seed (or public) data to be used in extraction
emnist_train, emnist_test = get_emnist_data()

# Run a model extraction attack
attack = ModelExtractionAttack(
    query=query_mnist,
    query_limit=100,
    victim_input_shape=(1, 1, 28, 28),
    victim_output_targets=10,
    substitute_input_shape=(1, 3, 28, 28),
    synthesizer="copycat",
    substitute_model_arch=ImagenetTransferLearning,
    substitute_input_size=1000,
    seed_data_train=emnist_train,
    seed_data_test=emnist_test,
    #callback=callback
)



GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33 K  
2 | layer_3 | Linear | 2 K   


/home/suhacker/.cache/pypoetry/virtualenvs/privacyraven-L5F_QRuO-py3.8/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The validation_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/home/suhacker/.cache/pypoetry/virtualenvs/privacyraven-L5F_QRuO-py3.8/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


/home/suhacker/.cache/pypoetry/virtualenvs/privacyraven-L5F_QRuO-py3.8/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The testing_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_loss': tensor(0.1019, device='cuda:0'),
 'test_loss': tensor(0.1019, device='cuda:0')}
--------------------------------------------------------------------------------

torch.Size([50, 1, 28, 28])
torch.Size([50, 10])
torch.Size([50, 1, 28, 28])
torch.Size([50, 10])
Synthesis complete
Synthetic Data Generated


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type        | Params
--------------------------------------------------
0 | feature_extractor | MobileNetV2 | 3 M   
1 | layer_1           | Linear      | 128 K 
2 | layer_2           | Linear      | 33 K  
3 | layer_3           | Linear      | 2 K   


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_loss': tensor(6.5600, device='cuda:0'),
 'test_loss': tensor(6.5600, device='cuda:0')}
--------------------------------------------------------------------------------

torch.Size([100, 3, 28, 28])
torch.Size([100, 10])
torch.Size([100, 1, 28, 28])
torch.Size([100, 10])
Out of 100 data points, the models agreed upon 4.


In [5]:
shadow = attack.substitute_model

In [6]:
from privacyraven.extraction.synthesis import process_data

In [7]:
x, y = process_data(emnist_train, 100)

In [8]:
from privacyraven.utils.query import query_model

pred, target = query_model(shadow, x, (1, 3, 28, 28))

torch.Size([100, 3, 28, 28])
torch.Size([100, 10])


In [16]:
import torch.nn as nn

# Toggle the losses 

loss = nn.KLDivLoss()
print(pred.size())
print(target.size())
print(y.size())
pred.cuda()
y.cuda()
output = loss(target, y)

# Can we calculate the loss as per a transfer based attack? 

torch.Size([100, 10])
torch.Size([100])
torch.Size([100])


/home/suhacker/.cache/pypoetry/virtualenvs/privacyraven-L5F_QRuO-py3.8/lib/python3.8/site-packages/torch/nn/functional.py:2352: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


In [17]:
output

tensor(-15.3290)